In [ ]:
import os
import json
print(os.getcwd())

# Data Cleaning

In [ ]:

WORKING_PATH = ""


In [ ]:
def load_data():
    data_set=dict()
    train_set = []
    val_set = []
    test_set = []
    for dataset in ["train"]:
        file=open(os.path.join(WORKING_PATH,"dataset_text/text_data/",dataset+".txt"),"rb")
        for line in file:
            content=eval(line)
            image=content[0]
            sentence=content[1]
            group=content[2]
            if os.path.isfile(os.path.join(WORKING_PATH,"dataset_image",image+".jpg")):
                train_set.append([content[0],content[1],content[2]])
    for dataset in ["valid"]:
        file=open(os.path.join(WORKING_PATH,"dataset_text/text_data/",dataset+".txt"),"rb")
        for line in file:
            content=eval(line)
            image=content[0]
            sentence=content[1]
            group=content[3] #2 hashtag
            if os.path.isfile(os.path.join(WORKING_PATH,"dataset_image",image+".jpg")):
                val_set.append([content[0],content[1],content[3],content[2]])
    for dataset in ["test"]:
        file=open(os.path.join(WORKING_PATH,"dataset_text/text_data/",dataset+".txt"),"rb")
        for line in file:
            content=eval(line)
            image=content[0]
            sentence=content[1]
            group=content[3] #2 is the 
            if os.path.isfile(os.path.join(WORKING_PATH,"dataset_image",image+".jpg")):
                test_set.append([content[0],content[1],content[3],content[2]])
    return train_set, val_set, test_set

def read_json(path):
    with open(path,"r",encoding = 'utf-8') as f:
        data = json.load(f)
    return data
def write_json(path,data):
    with open(path,"w",encoding = 'utf-8') as f:
        json.dump(data,f)

In [ ]:
import matplotlib.pyplot as plt # plt 用于显示图片
import os
from PIL import Image
import random

In [ ]:
train_set, val_set, test_set = load_data()


In [ ]:
def clean(dataset):
    dataset_ = []
    for sample in dataset:
        line = sample[1].strip()
        if "sarcasm" in line:
            continue
        if "sarcastic" in line:
            continue
        if "reposting" in line:
            continue
        if "<url>" in line:
            continue
        if "joke" in line:
            continue
        if "humour" in line:
            continue
        if "humor" in line:
            continue
        if "jokes" in line:
            continue
        if "irony" in line:
            continue
        if "ironic" in line:
            continue
        if "exgag" in line:
            continue
        sample[1] = line
        dataset_.append(sample)
    return dataset_

trainset = clean(train_set)
valset = clean(val_set)
testset = clean(test_set)

In [ ]:
# the final paths
write_json("./sarcasm/twitter/dataset_text/train.json", trainset)
write_json("./sarcasm/twitter/dataset_text/val.json", valset)
write_json("./sarcasm/twitter/dataset_text/test.json", testset)

# Visualization for twitter-image pair

In [ ]:
# i= random.randint(1,2000)
# print(423)
i=1604 
print(val[i][1])
print(val[i][2],val[i][3])
print(val[i][-3])
display_img(test[i][0])


In [ ]:
from PIL import Image
import os
import cv2
def display_img(flag):
    path = "./sarcasm/twitter/dataset_image"
    path = os.path.join(path,flag+".jpg")
    im=Image.open(path)
    im.show()

In [ ]:
print(len(train_set),len(val_set),len(test_set))

In [ ]:
test_dataset[423]

In [ ]:
img_switch = Image.open("./sarcasm/demo.png")

In [ ]:
img_switch = img_switch.resize((224,224)).convert('RGBA')

In [ ]:
b.paste(img_switch,(0,0,224,224),mask=a)

In [ ]:
r, g, b, a = img_switch.split()

In [ ]:
a = Image.new('RGB', (224, 224), (0,0,0)).convert('RGBA')

In [ ]:
w = img_switch.size[0]               #获取图片宽度
h = img_switch.size[1]               #获取图片高度
img_1 = img_switch.crop([2*w/3, 2*h/3, w, h])       #获取左上1/4的图片
img_1.save('demo_images/' + '8' + '.png')          #保存在本地图片命名为1.jpg
# img_2 = img.crop([w/2, 0, w, h/2])       #获得右上1/4的图片
# img_2.save('./' + '2' + '.jpg')          #保存在本地图片命名为2.jpg
# img_3 = img.crop([0, h/2, w/2, h])       #获取左下1/4的图片
# img_3.save('./' + '3' + '.jpg')          #保存在本地图片命名为3.jpg
# img_4 = img.crop([w/2, h/2, w, h])       #获取右下1/4的图片
# img_4.save('./' + '4' + '.jpg') 

In [ ]:
b=img_switch .load()

In [ ]:
# 改变图片被背景色
b=img_switch .load()
for y in range(224):
    for x in range(224):
        if all(b[x,y][i]>220 for i in range(4)):
            b[x,y] = 238,233,233


In [ ]:
img_1

In [ ]:
img_switch.save("demo.png")

# 图像特征抽取

In [ ]:
from PIL import Image
import torch
from torchvision import transforms
from pytorc import ViT

## VIT

In [ ]:
# 224 7*7 32*32
# use pre_trained model B_32 L_32 
model_name = 'B_32_imagenet1k'
model = ViT(model_name, pretrained=True)


In [ ]:
def B_32_embedding(dataset, img_path):
    model_name = 'B_32'
    model = ViT(model_name, pretrained=True)
    tfms = transforms.Compose([transforms.Resize(model.image_size), transforms.ToTensor(), 
                               transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),])
    print(model.image_size)
    embedding = []
    model.__delattr__("fc")
    model.cuda()
    model.eval()
    for sample in dataset:
        img = os.path.join(img_path,sample[0]+".jpg")
        img = Image.open(img)
        img = tfms(img).unsqueeze(0)
        with torch.no_grad():
            # remove class token
            img = model(img.cuda())[0,:-1,:]
        embedding.append(img.cpu())
    return embedding

def L_32_embedding(dataset, img_path):
    model_name = 'L_32'
    model = ViT(model_name, pretrained=True)
    tfms = transforms.Compose([transforms.Resize(model.image_size), transforms.ToTensor(), 
                               transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),])
    print(model.image_size)
    embedding = []
    model.__delattr__("fc")
    model.cuda()
    model.eval()
    for sample in dataset:
        img = os.path.join(img_path,sample[0]+".jpg")
        img = Image.open(img)
        img = tfms(img).unsqueeze(0)
        with torch.no_grad():
            # remove class token
            img = model(img.cuda())[0,:-1,:]
        embedding.append(img.cpu())
    return embedding
    

In [ ]:
# B_train_emb = B_32_embedding(trainset, img_path = "./sarcasm/twitter/dataset_image")
B_val_emb = B_32_embedding(valset, img_path = "./sarcasm/twitter/dataset_image")
B_test_emb = B_32_embedding(testset, img_path = "./sarcasm/twitter/dataset_image")

In [ ]:
L_train_emb = L_32_embedding(trainset, img_path = "./sarcasm/twitter/dataset_image")
L_val_emb = L_32_embedding(valset, img_path = "./sarcasm/twitter/dataset_image")
L_test_emb = L_32_embedding(testset, img_path = "./sarcasm/twitter/dataset_image")

In [ ]:
torch.save(L_train_emb, "./sarcasm/twitter/img_emb/train_L32.pt")
torch.save(L_val_emb, "./sarcasm/twitter/img_emb/val_L32.pt")
torch.save(L_test_emb, "./sarcasm/twitter/img_emb/test_L32.pt")

## Resnet

In [ ]:
import torchvision.models as models
resnet152 = models.resnet152(pretrained=True, progress = True)


In [ ]:
def Resnet152_embedding(dataset, img_path):
    resnet152 = models.resnet152(pretrained=True, progress = True)
    tfms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), 
                               transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    w=int(224/7)
    h=int(224/7)

    embedding = []
    resnet152.fc = torch.nn.Identity()
    resnet152.cuda()
    resnet152.eval()
    for sample in dataset:
        img = os.path.join(img_path,sample[0]+".jpg")
        img = Image.open(img)
        img = tfms(img).unsqueeze(0)
        patch = []
        for row in range(7):
            for col in range(7):
                patch.append(img[:,:,row*w:row*w+w,col*h:col*h+h])
        patches = torch.cat(patch, dim=0)
        with torch.no_grad():
            embedding.append(resnet152(patches.cuda()).cpu())
    return embedding

In [ ]:
train_res152 = Resnet152_embedding(trainset,img_path='./sarcasm/twitter/dataset_image' )

In [ ]:
torch.save(train_res152,"./sarcasm/twitter/img_emb/train_152.pt" )

In [ ]:
val_res152 = Resnet152_embedding(valset,img_path='./sarcasm/twitter/dataset_image' )

In [ ]:
torch.save(val_res152,"./sarcasm/twitter/img_emb/val_152.pt" )

In [ ]:
test_res152 = Resnet152_embedding(testset,img_path='./sarcasm/twitter/dataset_image' )

In [ ]:
torch.save(test_res152,"./sarcasm/twitter/img_emb/test_152.pt" )

# 依赖生成

In [ ]:
def re_organize_caption(tokens,noun_phrases):
    # tokens_smaple token+np
    # token_map token_map[a] a 是
    tokens_sample = []
    chunk_index = 0
    chunk_len = len(noun_phrases)
    i = 0
    token_map = []
    while (i<len(tokens)):
        if chunk_index<chunk_len:
            if i<noun_phrases[chunk_index][1]:
                tokens_sample.append(tokens[i][0])
                token_map.append(len(tokens_sample)-1)
                i = i+1
            else:
                tokens_sample.append(noun_phrases[chunk_index][0])
                for a in range(i,noun_phrases[chunk_index][2]):
                    token_map.append(len(tokens_sample)-1)
                i = noun_phrases[chunk_index][2]
                chunk_index = chunk_index+1
        else:
            tokens_sample.append(tokens[i][0])
            token_map.append(len(tokens_sample)-1)
            i = i+1
    return tokens_sample, token_map

In [ ]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [ ]:
a = "Men never were shown in a negative light" 


In [ ]:
doc = nlp(a)

In [ ]:
[(token.text.lower(),token.i,token.head.i, token.is_punct) for token in doc]

In [ ]:
# token dependency
def token_dependency(dataset):
    for i,sample in enumerate(dataset):
        dataset_article = {}
        dataset_article_a = {}
        doc = nlp(sample[1])
        dataset_article["token_caption"] = [(token.text.lower(),token.i,token.head.i, token.is_punct) for token in doc]
        dataset_article["chunk"] =  [(chunk.text.lower(),chunk.start,chunk.end) for chunk in doc.noun_chunks]
        token_sample,token_map = re_organize_caption(dataset_article["token_caption"],dataset_article["chunk"])
        dependency = [(token_map[t[1]],token_map[t[2]]) for t in dataset_article["token_caption"] if (not (token_map[t[1]]) == token_map[t[2]]) and 
                      (not t[3])]
        
        dataset_article_a["chunk_cap"] = token_sample
        dataset_article_a["token_cap"] = [t[0] for t in dataset_article["token_caption"]]
        
        dataset_article_a["token_dep"] = [(t[1],t[2]) for t in dataset_article["token_caption"] if (not t[1] == t[2]) and (not t[3]) and t[0]!=" " 
                                          and dataset_article["token_caption"][t[2]][0]!= " "]
        dataset_article_a["chunk_dep"] = dependency
        
        dataset_article_a["chunk"] = [temp[0] for temp in dataset_article["chunk"]]
        dataset_article_a["chunk_index"] = [token_map[temp[1]] for temp in dataset_article["chunk"]]
        
        dataset[i].append(dataset_article_a)
    return dataset
        
        
    

In [ ]:
trainset = read_json("./sarcasm/twitter/dataset_text/train.json")
valset = read_json("./sarcasm/twitter/dataset_text/val.json")
testset = read_json("./sarcasm/twitter/dataset_text/test.json")

In [ ]:
test_dep = token_dependency(testset)

In [ ]:
val_dep = token_dependency(valset)

In [ ]:
train_dep = token_dependency(trainset)

In [ ]:
write_json("./sarcasm/twitter/dataset_text/traindep.json", train_dep)
write_json("./sarcasm/twitter/dataset_text/valdep.json", val_dep)
write_json("./sarcasm/twitter/dataset_text/testdep.json", test_dep)

In [ ]:
len(train_dep)

# Knowledge

In [ ]:
# train的caption有一定的重复

In [ ]:
path_attr = "./sarcasm/twitter/dataset_text/multilabel_database/img_to_five_words.txt"
cap_test = "./sarcasm/twitter/dataset_text/gecaption/testcap.json"
cap_val = "./sarcasm/twitter/dataset_text/gecaption/valcap.json"
cap_train = "./sarcasm/twitter/dataset_text/gecaption/traincap.json"
train_dep = "./sarcasm/twitter/dataset_text/traindep.json"
val_dep = "./sarcasm/twitter/dataset_text/valdep.json"
test_dep = "./sarcasm/twitter/dataset_text/testdep.json"

In [ ]:
import json
def read_json(path):
    with open(path,"r",encoding = 'utf-8') as f:
        data = json.load(f)
    return data
def write_json(path,data):
    with open(path,"w",encoding = 'utf-8') as f:
        json.dump(data,f)

In [ ]:
train_dep = read_json(train_dep)
val_dep = read_json(val_dep)
test_dep = read_json(test_dep)

In [ ]:
cap_test = read_json(cap_test)
cap_val = read_json(cap_val)
cap_train = read_json(cap_train)

In [ ]:
print(len(train_dep), len(val_dep), len(test_dep))

In [ ]:
print(len(cap_train), len(cap_val), len(cap_test))

In [ ]:
data_attr=dict()
file=open("./sarcasm/twitter/dataset_text/multilabel_database/img_to_five_words.txt","rb")
a=0
for line in file:
    content=eval(line)
    data_attr[content[0]]=content[1:]

In [ ]:
# train_txt
file_path = "train.txt"
with open(file_path,"w") as f:
    for sample in train_dataset:
        f.write("dataset_image/"+sample[0]+".jpg\n")

In [ ]:
file_path = "test.txt"
with open(file_path,"w") as f:
    for sample in test_dataset:
        f.write("dataset_image/"+sample[0]+".jpg\n")

In [ ]:
file_path = "val.txt"
with open(file_path,"w") as f:
    for sample in val_dataset:
        f.write("dataset_image/"+sample[0]+".jpg\n")

In [ ]:
# add anp
test_anp= read_json("./sarcasm/twitter/dataset_text/anp/test.json")
val_anp= read_json("./sarcasm/twitter/dataset_text/anp/val.json")
train_anp= read_json("./sarcasm/twitter/dataset_text/anp/train.json")


In [ ]:
test_anp_set = []
for sample in test_anp['images']:
    anps = sample['bi-concepts']
    test_anp_set.append([a.replace('_', " ") for a in list(anps.keys())])  

In [ ]:
val_anp_set = []
for sample in val_anp['images']:
    anps = sample['bi-concepts']
    val_anp_set.append([a.replace('_', " ") for a in list(anps.keys())])  

In [ ]:
train_anp_set = []
for sample in train_anp['images']:
    anps = sample['bi-concepts']
    train_anp_set.append([a.replace('_', " ") for a in list(anps.keys())])  

In [ ]:
print(len(train_anp_set), len(val_anp_set), len(test_anp_set))

In [ ]:
train_dataset = []
for i, sample in enumerate(train_dep):
    id_attr = sample[0]
    sample.append(cap_train[20:][i])
    attr = data_attr[id_attr]
    sample.append(attr)
    sample.append(train_anp_set[i])
    train_dataset.append(sample)

In [ ]:
val_dataset = []
for i, sample in enumerate(val_dep):
    id_attr = sample[0]
    sample.append(cap_val[i])
    attr = data_attr[id_attr]
    sample.append(attr)
    sample.append(val_anp_set[i])
    val_dataset.append(sample)

In [ ]:
test_dataset = []
for i, sample in enumerate(test_dep):
    id_attr = sample[0]
    sample.append(cap_test[i])
    attr = data_attr[id_attr]
    sample.append(attr)
    sample.append(test_anp_set[i])
    test_dataset.append(sample)

In [ ]:
write_json("./sarcasm/twitter/dataset_text/trainknow.json", train_dataset)
write_json("./twitter/dataset_text/valknow.json", val_dataset)
write_json("./sarcasm/twitter/dataset_text/testknow.json", test_dataset)

In [ ]:
train = read_json("./sarcasm/twitter/dataset_text/trainknow.json")
val = read_json("./sarcasm/twitter/dataset_text/valknow.json")
test = read_json("./sarcasm/twitter/dataset_text/testknow.json")
# *.

In [ ]:
test[0]

In [ ]:
for i,sample in enumerate(val):
    print (i, sample[-3])

In [ ]:
isinstance(test_dataset[0][-1], list)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_trf")
def re_organize_caption(tokens,noun_phrases):
    # tokens_smaple token+np
    # token_map token_map[a] a 是
    tokens_sample = []
    chunk_index = 0
    chunk_len = len(noun_phrases)
    i = 0
    token_map = []
    while (i<len(tokens)):
        if chunk_index<chunk_len:
            if i<noun_phrases[chunk_index][1]:
                tokens_sample.append(tokens[i][0])
                token_map.append(len(tokens_sample)-1)
                i = i+1
            else:
                tokens_sample.append(noun_phrases[chunk_index][0])
                for a in range(i,noun_phrases[chunk_index][2]):
                    token_map.append(len(tokens_sample)-1)
                i = noun_phrases[chunk_index][2]
                chunk_index = chunk_index+1
        else:
            tokens_sample.append(tokens[i][0])
            token_map.append(len(tokens_sample)-1)
            i = i+1
    return tokens_sample, token_map

In [ ]:
# token dependency
def token_dependency(dataset):
    for i,sample in enumerate(dataset):
        dataset_article = {}
        dataset_article_a = {}
        # the caption index is -3 
        doc = nlp(sample[-3].strip())
        dataset_article["token_caption"] = [(token.text.lower(),token.i,token.head.i, token.is_punct) for token in doc]
        dataset_article["chunk"] =  [(chunk.text.lower(),chunk.start,chunk.end) for chunk in doc.noun_chunks]
        token_sample,token_map = re_organize_caption(dataset_article["token_caption"],dataset_article["chunk"])
        dependency = [(token_map[t[1]],token_map[t[2]]) for t in dataset_article["token_caption"] if (not (token_map[t[1]]) == token_map[t[2]]) and 
                      (not t[3])]
        
        dataset_article_a["token_cap"] = [t[0] for t in dataset_article["token_caption"]]
        
        dataset_article_a["token_dep"] = [(t[1],t[2]) for t in dataset_article["token_caption"] if (not t[1] == t[2]) and (not t[3]) and t[0]!=" " 
                                          and dataset_article["token_caption"][t[2]][0]!= " "]
        
        dataset[i].append(dataset_article_a)
    return dataset

In [ ]:
test_dep = token_dependency(test)

In [ ]:
val_dep = token_dependency(val)

In [ ]:
train_dep = token_dependency(train)

In [ ]:
# write_json("./sarcasm/twitter/dataset_text/trainknow_dep.json", train_dep)
write_json("./sarcasm/twitter/dataset_text/valknow_dep.json", val_dep)
# write_json("./sarcasm/twitter/dataset_text/testknow_dep.json", test_dep)

In [ ]:
# train_dep = read_json("./sarcasm/twitter/dataset_text/trainknow_dep.json")
val_dep = read_json("./sarcasm/twitter/dataset_text/valknow_dep.json")
# test_dep = read_json("./sarcasm/twitter/dataset_text/testknow_dep.json")

In [ ]:
length = []
for smaple in val_dep:
    length.append(len(smaple[-1]['token_dep'])) 

In [ ]:
min(length)